In [4]:
# Hack Sample Code
# This shows examples of logging in and using the subreddit, submission, redditor, ancd comments objects
# It is not implied that you need to collect these specific attributes of submissions or redditors, but are merely examples

# Imports
import praw
import pandas as pd
import numpy as np
import datetime


In [11]:

# Login and create Reddit instance
myClientIDvar = '7H-wkx3wmj96hJazvWQb-A'
myClientSecret = 'kKTtoMJA0gYXEr9d0ohMp-niRM0yhw'
myRedditUserName = 'Legitimate_Thing5210'
reddit = praw.Reddit(client_id=myClientIDvar, client_secret=myClientSecret, user_agent=myRedditUserName)


In [35]:
"""
Get the top 5 posts on Christianity
"""
subredditName = 'christianity'
subreddit_top_5 = reddit.subreddit(subredditName).hot(limit=5)

In [37]:
""""
Get Sample Dataframe of top 5 Posts' information
"""
post_lists = []

for indexPosts, submission in enumerate(subreddit_top_5):
    post_lists.append({
      'id': submission.id,
      'Sub': subredditName,
      'Title': submission.title,
      'URL': submission.permalink,
      'Time': datetime.datetime.fromtimestamp(submission.created_utc),
      'Author': str(submission.author),
      'Body': submission.selftext,
      'IsSelfPost': bool(submission.is_self)
    })

    print(submission.is_self)
    print('Retrieved data for '+str(indexPosts+1)+' posts')


dfPosts = pd.DataFrame(post_lists)
print(dfPosts)

Empty DataFrame
Columns: []
Index: []


In [31]:
"""
Get Sample Dataframe of author information
"""

author_list = []
for indexAuthor, thisIndex in enumerate(dfPosts.index):
    # Get Entry Information
    thisSubmission = reddit.submission(id=dfPosts.loc[thisIndex,'Unique'])
    thisAuthor = thisSubmission.author

    # Get Author Information
    TempDictAuthor = {}

    # Verifies that author exists and has not been suspended (as suspended accounts do not have most attributes)
    if ((thisSubmission.author is not None) and hasattr(thisAuthor,'created_utc')): 
        TempDictAuthor['userName'] = thisAuthor.name
        TempDictAuthor['accountAge'] = thisAuthor.created_utc
        TempDictAuthor['commentKarma'] = thisAuthor.comment_karma
        TempDictAuthor['linkKarma'] = thisAuthor.link_karma
        TempDictAuthor['isMod'] = thisAuthor.is_mod
        numPosts = 0
        numCommentsList = []

        # Get number of posts by the author in the past month and the average number of comments to it
        for thisAuthorPost in thisAuthor.submissions.top(time_filter="month"):
            numPosts = numPosts + 1
            numComments = 0
            for tempComment in thisAuthorPost.comments:
                if hasattr(tempComment,'is_submitter'): # If tempComment does not have a is_submitter attribute, it is because it is a "More Comments" link
                    numComments = numComments + 1
                else:
                    numComments = np.nan # For simplicity, just made this NaN as the summation is unknown unless clicking "More Comments", which is not part of this example code
            numCommentsList.append(numComments)
        TempDictAuthor['numPosts'] = numPosts
        TempDictAuthor['avgCommentsToPost'] = np.mean(numCommentsList)
    author_list.append(TempDictAuthor) # Append to dataframe

    print('Retrieved data for '+str(indexAuthor+1)+' authors')

dfAuthors = pd.DataFrame(author_list)
dfAuthors['isMod'] = dfAuthors['isMod'].astype('bool')
print(dfAuthors)

KeyError: 'isMod'